In [3]:
import os
#02_prog_starwar
#SWRT.txt
# базовая дирректория на моем компьютере
#base_path = '/home/gatupov/Рабочий стол/ignis_prog/starwars'
base_path = '/home/gatupov/Рабочий стол/ignis_prog/electro_city'
os.chdir(base_path)
###
#path_pic='/home/gatupov/Рабочий стол/ignis_prog/starwars/starwars_pic/' ## ОБЯЗАТЕЛЬНО ПОМЕНЯТЬ ПОЛЬЗОВАТЕЛЮ!!!!!!
path_pic='/home/gatupov/Рабочий стол/ignis_prog/electro_city/electro_city_pic/'
print('Проверьте, есть ли в вашей папке папка с картинками и файл меток \n')
print('ВВедите имя будущей папки для реквизита. Помните, что оно должно начинаться с  XX_prog_')
print('где XX - порядковый  номер программы у пойки')
name_file_out=input()
print('Введите имя дорожки отметок')
name_file=input()

Проверьте, есть ли в вашей папке папка с картинками и файл меток 

ВВедите имя будущей папки для реквизита. Помните, что оно должно начинаться с  XX_prog_
где XX - порядковый  номер программы у пойки
19_prog_electro
Введите имя дорожки отметок
retro_city_metki


In [6]:
from PIL import Image, ImageDraw
import shutil
import os
import collections
from math import floor
import sys

# удаление ненужных символов из строки в файле
def rems(line):
    line=line.replace(' ','')
    line=line.replace('\t','')
    line=line.replace('\r','')
    line=line.replace('\n','')
    return line

# создание дирректорий для разных участников (поек)
def direct(name_file):
    path=os.getcwd() #dir
    #path+='/ignis'
    path+='/'+name_file.split('.')[0]
    try:
        os.mkdir(path) # только на один уровень ниже. иначе исп. makedirs()
    except OSError:
        print('Дирректория  %s уже существует. Пересоздать? \n \
              (Если да, просто нажмите Enter. Если нет, перед этим введите что-нибудь в поле)' % path)
        flag_rm = str(input())
        if flag_rm == '':
            shutil.rmtree(path)
            os.mkdir(path)
            print('Дирректория  %s пересоздана\n' % path)
        else:
            print('Окей, прерываем программу. Видимо, попытались пересоздать не ту дирректорию.')
            sys.exit()
    else:
        print('Успешно создана дирректория %s' % path)
    return path

#  сохранение картинок с высостой 32 в нужные дирректории

def ren32(path_pic,path,num,metka):
    try:
        image=Image.open(path_pic+metka+'.bmp')
        image.save(path+'/'+'{:0>2}_{}.bmp'.format(num,metka), "BMP")
    except FileNotFoundError:
        print()
        print('НЕТ ФАЙЛА %s \n' %(path_pic+metka+'.bmp'))

#  сохранение картинок с высостой 64 в нужные дирректории       
def ren64(path_pic,path,num,metka):
    image1 = Image.open(path_pic+metka+'.bmp') #Открываем изображение. 
    image2=Image.open(path_pic+'white_64.bmp')
    width = image1.size[0] #Определяем ширину. 
    height1 = image1.size[1] #Определяем высоту.
    #print(path_pic+metka+'.bmp')
    height2 = image2.size[1] #Определяем высоту.
    draw = ImageDraw.Draw(image2) #Создаем инструмент для рисования. 
    pix = image1.load()
    #print(pix)

    for i in range(width):
        k=-1
        for j in range(height1):
            a = pix[i, j][0]
            b = pix[i, j][1]
            c = pix[i, j][2]
            k+=1
            draw.point((i, k), (a, b, c))
            k+=1
            draw.point((i, k), (a, b, c))

    image2.save(path+'/'+'{:0>2}_{}.bmp'.format(num,metka), "BMP")
    del draw
    
SYSTEM_SIZE = 400 # стандартная ширина картинки, для которой будет вычисляться задержка по времени!

### РАБОТА С ДИРРЕКТОРИЯМИ
# Вывод имени папки
print('Имя папки для реквизита')
try:
    print(name_file_out)
    print()
except NameError:
    print('Выполните 1 ячейку! Не задано имя папки для реквизита')
    sys.exit()
## создание файла _intervals в основной ignis папке, в котором помимо абс тайминга есть еще и относительный
path_intervals=os.getcwd()  #d
#path_intervals+='/ignis'
path_intervals+='/'+name_file_out.split('.')[0]+'_intervals.txt'
file_intervals=open(path_intervals,'w')

#создание дирректории для пойки. В name_path она будет храниться
name_path=direct(name_file_out) #d
file_out=open(name_path+'/program.txt','w') # создание program.txt в папке для пойки

# проверка и вывод имени дорожки отметок
print('Имя файла дорожки отметок')
try:
    print(name_file)
except NameError:
    print('Выполните 1 ячейку! не задано имя дорожки отметок')
    sys.exit()

# ОСНОВНЫЕ МАССИВЫ ##############################################
###### Чтобы понять, проще на них посмотреть. Они выводятся в конце работы программы

time=[] # для абс времени каждой метки в формате файла меток audacity
metka=[] # для имен картинок для каждой метки: запись нескольких через ; Картинки повторяются
person=[] # для имен поек для каждой метки: запись нескольких через ;. Имена повторяются
full_metki=[] # массив из полного текста (без изменений, как файле) каждой метки
rekv=[] # высота реквизита (32 или 64)
person_rekv=[] # имя пойки
flag_rekv=0 # количество поек (точнее программ для поек) в шоу
#########################################################################




# ОБРАБОТКА ФАЙЛА ОТМЕТОК ############################################################
with open(name_file) as mfile:
    for st in mfile: # построчная обработка
        if not(len(st.split('\t'))==3) and not('#' in st): # проверка, нет ли какой фигни в виде опечаток.
            print('\nDANGER. not 3 columns in row in file with time:')
            print(st+'\n')
        else:
            # обработка строк с #
            if ('#' in st):
                line=st.split('-')[1]
                line=rems(line) # удаление невидимых символов
                rekv.append(int(line))  # сохранение высоты реквизита (32 или 64)
                
                line=st.split('-')[0]
                line=line.replace('#','')
                line=rems(line) # удаление невидимых символов
                person_rekv.append(line) # сохранение имени пойки
                flag_rekv+=1
            # обработка обычных строк с таймингом
            if not('#' in st):   
                time.append(st.split('\t')[1]) # абс время
                s=st.split('\t')[2]
                full_metki.append(s) # полный текст метки
                # обработка текста метки s
                if '>' in s: # если сразу несколько картинок для 1 тайминга
                    if '/' in s:
                        chasti=s.split('/')
                        person_sbor=''
                        metka_sbor=''
                        for ch in chasti:
                            person_sbor+=ch.split('>')[0]+';'
                            metka_sbor+=ch.split('>')[1]+';'
                        person.append(person_sbor[:-1]) # запись всех имен поек для танного тайминга через ;
                        metka.append(metka_sbor[:-2]) # запись всех картинок поек для данного тайминга через ;
                    else:
                        person.append(s.split('>')[0])
                        metka.append(s.split('>')[1][:-1])
                else: # если одна картинка для 1 тайминга
                    person.append('all')
                    metka.append(s[:-1])
##################################################################################################

### Проверка, все ли картинки из файла меток есть в папке
list_pictures = [] # список картинок из файла меток
for el in metka:
    for pc in el.split(';'):
        list_pictures.append(pc)
list_pictures = set(list_pictures)
list_pictures = list(list_pictures)

    # получение названия всех картинок
pictures_in_dir = os.listdir(path_pic)
print(pictures_in_dir)
for i in range(len(pictures_in_dir)):
    pictures_in_dir[i] = pictures_in_dir[i].split('.')[0] # избавляемся от .bmp

fl_picmetka = 0
fl_picend = 0
for pic_metka in list_pictures:
    pic_metka = pic_metka.replace('*','')
    pic_metka = pic_metka.replace('$','')
    if not(pic_metka in pictures_in_dir) and not(pic_metka=='end'):
        fl_picmetka = 1
        print('НЕТ ФАЙЛА {}'.format(pic_metka))
    if pic_metka == 'end':
        fl_picend = 1
if fl_picend == 0:
    print('В файле меток отсутствует метка выключения end!')

if fl_picmetka ==1 or fl_picend == 0:
    sys.exit()
        
    

# получение списка из НЕПОВТОРЯЮЩИХСЯ имен поек
all_persons=[] 
for elem in person:
    pers=elem.split(';')
    for p in pers:
        all_persons.append(p)

persons=list(collections.Counter(all_persons).keys()) # список из НЕПОВТОРЯЮЩИХСЯ имен поек persons
rekv2=[int(32)]*len(persons) # Пока у всех высота реквизита 32.

## Получение реальной высоты реквизита для каждой пойки. Будет храниться в rekv2
if flag_rekv>1:
    for j in range(len(persons)):
        for i in range(len(person_rekv)):
            if person_rekv[i]==persons[j]:
                rekv2[j]=rekv[i]
if flag_rekv==1:
    for j in range(len(persons)):
        rekv2[j]=rekv[0]



people=0 # флаг для количества поек

# Если только одна прога people=1, если их сразу несколько, people = 0
if len(persons)==1 and persons[0]=='all':
    people=1 # одна прога на всех
    if flag_rekv==0:
        print("Ошибка: вы не указали длину реквизита в пикселях в файла меток")
        sys.exit()

# Вычисляем абсолютное время относительно точки включения ###############################
dtime=[] # абсолютное время относительно точки включения
stime=[] # абсолютное время относительно точки включения в нужном для e-poi формате
pictures=[]
intervals=[] # относительное время для метки для файла _intervals. Не особо нужно 
pop=0 # для учета секундной задержки
for j in range(len(time)):
    dtime.append(( float(time[j]) - float(time[0]) ))
for j in range(1,len(time)):
    intervals.append( ( float(time[j]) - float(time[j-1]) ) )
intervals.append(0)
for j in range(len(time)):
    minn=int(dtime[j]//60)
    sec=int(floor(dtime[j]%60))-pop # минус секунда! Учет секундной задержки. Но не для первого тайминга
    if (sec==-1):
        sec=59
        minn-=1
    msec=int(floor((dtime[j] - floor(dtime[j]%60) - minn*60)*100))
    stime.append('{:0>2}:{:0>2}:{:0>2}'.format(minn,sec,msec))
    pop=1

### вставка для перевода в формат ignis поев. непонятно.
#print(stime)
#file_ignis=open('ignis.txt','w')
#file_ignis.write('Picture ID;Duration\n')
#for j in range(len(full_metki)):
    #file_ignis.write('{}\t{}\t;{}\n'.format(full_metki[j][:-1],stime[j],round(intervals[j],2)))
    #file_intervals.write('{}\t{}\t{}\n'.format(full_metki[j][:-1],round(intervals[j],2),stime[j]))

#print('\n #### ЛОГИ #### \n')
#dop='Finish - {}\nRepeat after finish - no\nLock buttons - yes'.format(stime[len(time)-1])

######## Наполнение папок и создания файла программы: Если одна программа в файле меток
if people==1:
    if flag_rekv==0:
        print("Ошибка: вы не указали длину реквизита в пикселях в начале файла меток")
        sys.exit()
    
    
    numer=0
    for j in range(len(time)):
        star=0 # наличие * в строке
        dol = 0 # наличие $ в строке
        if '*' in metka[j]:
            metka[j] = metka[j].replace('*','')
            star=1
        if '$' in metka[j]:
            metka[j] = metka[j].replace('$','')
            dol=1
        
        if metka[j]=='end': # точка выключения
            dop='Finish - {}\nRepeat after finish - no\nLock buttons - yes'.format(stime[j])
            file_out.write(dop)
        else:
            if not metka[j] in pictures: # если картинка еще не была сохранена в нужной папка (нет в pictures)
                numer+=1
                if rekv[0]==32:
                    ren32(path_pic,name_path,numer,metka[j]) # сохранение картинки
                if rekv[0]==64:
                    ren64(path_pic,name_path,numer,metka[j]) # сохранение картинки
                if not(rekv[0]==32) and not(rekv[0]==64): # проверка
                    print('Описка в указании длины реквизита. Неверное число пикселей')
                    print('Допустимое число пикселей - 32 или 64')
                # Нужно ли менять скорость показа картинки (в () указывается задержка)
                if star==0 and dol==0: # не нужно
                    file_out.write('{:0>2}_{} - {}\n'.format(numer,metka[j],stime[j]))
                if star==1 and dol==0: # нужно
                    speed=round((intervals[j]/SYSTEM_SIZE*1000),1) # задержка 
                    if speed > 9.9: # нужно поменять величину картинки
                        print('\nРазмер картинки {} слишком мал. Ее длительность {} c. Время {}'.format(metka[j],intervals[j],stime[j]))
                        print('Создайте картинку и в метках укажите знак $ \n 1 секунта = 101 пиксель\n')
                        file_out.write('{:0>2}_{} - {} ({})\n'.format(numer,metka[j],stime[j],speed))
                    if speed < 0.1: # нужно поменять величину картинки
                        print('\nРазмер картинки {} слишком велик. Ее длительность {} c. Время {}'.format(metka[j],intervals[j],stime[j]))
                        print('Создайте картинку и в метках укажите знак $ \n 1 секунта = 101 пиксель\n')
                        file_out.write('{:0>2}_{} - {} ({})\n'.format(numer,metka[j],stime[j],speed))
                    if speed >= 0.1 and speed <= 9.9:
                        file_out.write('{:0>2}_{} - {} ({})\n'.format(numer,metka[j],stime[j],speed)) #вычисляется прогой
                if star==0 and dol==1: # ширина картинки уже подрегулирована, на это указывает доллар
                    speed=9.9
                    file_out.write('{:0>2}_{} - {} ({})\n'.format(numer,metka[j],stime[j],speed))
                if star==1 and dol==1: # проверка
                    print('Ошибка в указаниии метки {}. Одновременно есть * и $'.format(metka[j]))
                    file_out.write('{:0>2}_{} - {} \n'.format(numer,metka[j],stime[j])) # запись в programm.txt
                pictures.append(metka[j]) # добавление в список сохраненных картинок
            else:
                for p in range(len(pictures)): # если картинка уже была сохранена находим ее индекс в массиве
                    if metka[j]==pictures[p]:
                        break
                #file_out.write('{:0>2}_{} - {}\n'.format(p+1,metka[j],stime[j]))
                # нужно ли менять скорость? Код такой же, как выже.
                if star==0 and dol==0:
                    file_out.write('{:0>2}_{} - {}\n'.format(p+1,metka[j],stime[j]))
                if star==1 and dol==0:
                    speed=round((intervals[j]/SYSTEM_SIZE*1000),1)
                    if speed > 9.9:
                        print('\nРазмер картинки {} слишком мал. Ее длительность {} c Время {}'.format(metka[j],intervals[j],stime[j]))
                        print('Создайте картинку и в метках укажите знак $ \n 1 секунта = 101 пиксель\n')
                        speed=9.9
                        file_out.write('{:0>2}_{} - {} ({})\n'.format(p+1,metka[j],stime[j],speed))
                    if speed < 0.1:
                        print('\nРазмер картинки {} слишком велик. Ее длительность {} c. Время {}'.format(metka[j],intervals[j],stime[j]))
                        print('Создайте картинку и в метках укажите знак $ \n 1 секунта = 101 пиксель\n')
                        speed=0.1
                        file_out.write('{:0>2}_{} - {} ({})\n'.format(p+1,metka[j],stime[j],speed))
                    if speed >= 0.1 and speed <= 9.9:
                        file_out.write('{:0>2}_{} - {} ({})\n'.format(p+1,metka[j],stime[j],speed))
                if star==0 and dol==1:
                    speed=9.9
                    file_out.write('{:0>2}_{} - {} ({})\n'.format(p+1,metka[j],stime[j],speed))
                if star==1 and dol==1:
                    print(' \nОШИБКА в указаниии метки {}. Одновременно есть * и $\n'.format(metka[j]))
                    file_out.write('{:0>2}_{} - {} \n'.format(p+1,metka[j],stime[j]))
    
    #file_out.write(dop)
    file_out.close()
    print('Записано в '+ name_file_out)
    print()

######## Наполнение папок и создания файла программы: Если несколько программ в файле меток
if people==0:
    person1=[] # промежуточные массивы
    rekv3=[]
    for t in range(len(persons)):
        if not persons[t]=='all':
            person1.append(persons[t])
            rekv3.append(rekv2[t])
    persons=person1 # здесь названия програм
    rekv=rekv3 # здесь высота реквизита для никх
    file_out.write('Ищи в других файлах с номерами') # костыль. Запись в program.txt
    file_out.close()
    per_n=-1
    for per in persons: # Делаем отдельно для каждой пойки (программы пойки)
        kombo_pictures=[] # массив для хранения картинок без повторения
        per_n+=1
        numer=0
        name_path=direct(name_file_out+'_'+per) # создание папки под конкретного человека (пойку)
        file_out=open(name_path+'/program.txt','w') # помещение туда program.txt
        for j in range(len(time)): # пробегаемся по всем таймингам
            if metka[j]=='end':
                dop='Finish - {}\nRepeat after finish - no\nLock buttons - yes'.format(stime[j])
                file_out.write(dop)
            else:
                flag_m=0
                kombo_metka=metka[j].split(';') # массив для картинок одного тайминга
                kombo_person=person[j].split(';') # массив для названий поек одного тайминга
                for m in range(len(kombo_metka)): # аналогично коду выже. Есть ли * и $
                    star=0
                    dol = 0
                    if '*' in kombo_metka[m]:
                        kombo_metka[m] = kombo_metka[m].replace('*','')
                        star=1
                    if '$' in kombo_metka[m]:
                        kombo_metka[m] = kombo_metka[m].replace('$','')
                        dol=1

                    if per==kombo_person[m] or person[j]=='all': # если в данной метке есть название пойки или all
                        if not kombo_metka[m] in kombo_pictures: # если эта картинка еще не сохранялась в нужной папке
                            # то сохраняем. Аналогично коду для people=1
                            numer+=1
                            if rekv[per_n]==32:
                                ren32(path_pic,name_path,numer,rems(kombo_metka[m]))
                            if rekv[per_n]==64:
                                ren64(path_pic,name_path,numer,rems(kombo_metka[m]))
                            if not(rekv[per_n]==32) and not(rekv[per_n]==64):
                                print('Описка в указании длины реквизита. Неверное число пикселей')
                                print('Допустимое число пикселей - 32 или 64')
                            
                            #file_out.write('{:0>2}_{} - {}\n'.format(numer,kombo_metka[m],stime[j]))
                            # определение задержки, если нужно. Аналогично коду для people=1
                            if star==0 and dol==0:
                                file_out.write('{:0>2}_{} - {}\n'.format(numer,rems(kombo_metka[m]),stime[j]))
                            if star==1 and dol==0:
                                speed=round((intervals[j]/SYSTEM_SIZE*1000),1)
                                if speed > 9.9:
                                    print('\nРазмер картинки {} слишком мал. Ее длительность {} c. Время {}'.format(kombo_metka[m],intervals[j],stime[j]))
                                    print('Создайте картинку и в метках укажите знак $ \n 1 секунта = 101 пиксель\n')
                                    speed=9.9
                                if speed < 0.1:
                                    print('\nРазмер картинки {} слишком велик. Ее длительность {} c. Время {}'.format(kombo_metka[m],intervals[j],stime[j]))
                                    print('Создайте картинку и в метках укажите знак $ \n 1 секунта = 101 пиксель\n')
                                    speed=0.1
                                file_out.write('{:0>2}_{} - {} ({})\n'.format(numer,rems(kombo_metka[m]),stime[j],speed))
                            if star==0 and dol==1:
                                speed=9.9
                                file_out.write('{:0>2}_{} - {} ({})\n'.format(numer,rems(kombo_metka[m]),stime[j],speed))
                            if star==1 and dol==1:
                                print('Ошибка в указаниии метки {}. Одновременно есть * и $'.format(metka[j]))
                                file_out.write('{:0>2}_{} - {} \n'.format(numer,rems(kombo_metka[m]),stime[j]))
                            flag_m=1
                            kombo_pictures.append(kombo_metka[m])
                        else: # если уже сохраняли - просто записываем в program.txt. Аналогично коду для people=1
                            for p in range(len(kombo_pictures)):
                                if kombo_metka[m]==kombo_pictures[p]:
                                    break
                            # нужна ли задержка? Аналогично коду для people=1
                            if star==0 and dol==0:
                                file_out.write('{:0>2}_{} - {}\n'.format(p+1,rems(kombo_metka[m]),stime[j]))
                            if star==1 and dol==0:
                                speed=round((intervals[j]/SYSTEM_SIZE*1000),1)
                                if speed > 9.9:
                                    print('\nРазмер картинки {} слишком мал. Ее длительность {} c. Время {}'.format(kombo_metka[m],intervals[j],stime[j]))
                                    print('Создайте картинку и в метках укажите знак $ \n 1 секунта = 101 пиксель\n')
                                    speed=9.9
                                if speed < 0.1:
                                    print('\nРазмер картинки {} слишком велик. Ее длительность {} c. Время {}'.format(kombo_metka[m],intervals[j],stime[j]))
                                    print('Создайте картинку и в метках укажите знак $ \n 1 секунта = 101 пиксель\n')
                                    speed=0.1
                                file_out.write('{:0>2}_{} - {} ({})\n'.format(p+1,rems(kombo_metka[m]),stime[j],speed))
                            if star==0 and dol==1:
                                speed=9.9
                                file_out.write('{:0>2}_{} - {} ({})\n'.format(p+1,rems(kombo_metka[m]),stime[j],speed))
                            if star==1 and dol==1:
                                print('Ошибка в указаниии метки {}. Одновременно есть * и $'.format(metka[j]))
                                file_out.write('{:0>2}_{} - {} \n'.format(p+1,rems(kombo_metka[m]),stime[j]))
                            flag_m=1
                if flag_m==0: # если ни all, ни имя пойки не встретились - значит пойка не должна гореть,
                    # начиная с этого тайминга. Сохраняем картинку blackout (черный цвет) и записываем ее 
                    # в program.txt соответствующей пойки. Или не сохраняем, если blackout уже есть в папке
                    if not 'blackout' in kombo_pictures: # сохраняем blackout
                        numer+=1
                        if rekv[per_n]==32:
                            ren32(path_pic,name_path,numer,'blackout')
                        if rekv[per_n]==64:
                            ren64(path_pic,name_path,numer,'blackout')
                        if not(rekv[per_n]==32) and not(rekv[per_n]==64):
                            print('Описка в указании длины реквизита. Неверное число пикселей')
                            print('Допустимое число пикселей - 32 или 64')
                        file_out.write('{:0>2}_{} - {}\n'.format(numer,'blackout',stime[j]))
                        kombo_pictures.append('blackout')
                    else: # blackout уже есть
                        for p in range(len(kombo_pictures)):
                                if 'blackout'==kombo_pictures[p]:
                                    break
                        file_out.write('{:0>2}_{} - {}\n'.format(p+1,'blackout',stime[j]))
        file_intervals.close()                
        file_out.close()
        #file_ignis.close()
        print('Записано в '+name_file_out+'_'+per)
        print()
        


print()
print('Техническая информация для программистов')
print('metka')
print(metka)
print('time')
print(time)
print('person_rekv')
print(person_rekv)
print('rekv')
print(rekv)     

Имя папки для реквизита
19_prog_electro

Дирректория  /home/gatupov/Рабочий стол/ignis_prog/electro_city/19_prog_electro уже существует. Пересоздать? 
               (Если да, просто нажмите Enter. Если нет, перед этим введите что-нибудь в поле)

Дирректория  /home/gatupov/Рабочий стол/ignis_prog/electro_city/19_prog_electro пересоздана

Имя файла дорожки отметок
retro_city_metki
['galka.bmp', 'grey.bmp', 'lyel.bmp', 'ryg.bmp', 'ygas.bmp', 'bl.bmp', 'bwg.bmp', 'star_fill.bmp', 'black.bmp', 'hex.bmp', 'blackout.bmp', 'yel_strob.bmp', 'star_fill2.bmp', 'ind.bmp', 'pi.bmp', 'bl13.bmp', 'gold.bmp', 'star_fill1.bmp', 'bl23.bmp', 'star.bmp', 'red.bmp', 'yel.bmp', 'fire.bmp']
Дирректория  /home/gatupov/Рабочий стол/ignis_prog/electro_city/19_prog_electro_m уже существует. Пересоздать? 
               (Если да, просто нажмите Enter. Если нет, перед этим введите что-нибудь в поле)

Дирректория  /home/gatupov/Рабочий стол/ignis_prog/electro_city/19_prog_electro_m пересоздана

Записано в 19_prog_